In [1]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from semantic_text_similarity.models import WebBertSimilarity
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy import spatial
import parmap
import os
import swifter
from tqdm import tqdm

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [3]:
path = './data/bbc/2015/transcripts'
files = os.listdir(path)

data = []
for source in files:
    data.append(pd.read_csv(os.path.join(path, source)))
df = pd.concat(data)
df = df.drop(['Unnamed: 0', 'Has Transcript', 'Unavailable link', 'Unavailable reason'], axis=1)
df = df.reset_index(drop=True)
df

,Source,Date,Program Name,Time,Duration,Transcript
0,BBC1 London,5-jul-2015,BBC News,18:35,15 mins,# You’re stayin’ alive Stayin’ alive # Feel th...
1,BBC1 London,5-jul-2015,BBC News,22:20,20 mins,Greece has voted overwhelmingly to reject the ...
2,BBC1 London,4-jul-2015,BBC London News,17:10,5 mins,"Thank wu warg nwuch. Formula One now, and Lewi..."
3,BBC1 London,4-jul-2015,BBC News,22:20,20 mins,# Hold back the river Let me look in your eyes...
4,BBC1 London,7-jul-2015,BBC News at Six,18:00,30 mins,Ginger Baker is one of the most innovative dru...
...,...,...,...,...,...,...
1356,BBC1 London,30-jan-2015,BBC News at One,13:00,40 mins,"Ashley was to walk down here now, I’d justjump..."
1357,BBC News 24,30-jan-2015,BBC News at Six,18:00,30 mins,"Ten minutes! ..Six and a half! Sewers, you hav..."
1358,BBC1 London,30-jan-2015,BBC News at Ten,22:00,25 mins,The Church of England and the Catholic Church ...
1359,BBC1 London,25-jan-2015,BBC Weekend News,13:00,15 mins,20 years of planning. 10 years experimenting. ...


In [4]:
def filter_short_sentence(sentences):
    n_sent = len(sentences)
    filtered_sentences = []
    for i in range(n_sent):
        if len(sentences[i].split()) < 10:
            filtered_sentences.append(sentences[i])
    return filtered_sentences

def get_short_sentences(transcript):
    cluster = []
    sentences = nltk.sent_tokenize(transcript)
    sentences = filter_short_sentence(sentences)
    
    return sentences

In [5]:
transcripts = df['Transcript'].values

In [6]:
short_sentences = []
for transcript in tqdm(transcripts):
    short_sentences.extend(get_short_sentences(transcript))

100%|██████████| 1361/1361 [00:15<00:00, 87.18it/s] 


In [9]:
short_sentences[2]

'I don’t want this to be a show.'

In [10]:
with open('dropped_sentences_2015.txt', 'w') as f:
    f.writelines(short_sentences)